In [4]:
!pip3 install BeautifulSoup4
!pip3 install pandas

In [5]:
import csv
import requests
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup

In [6]:
URLs = {
    'SPORTS':'https://www.diaridegirona.cat/esports',
    'ECONOMIA':'https://www.diaridegirona.cat/economia/',
    'CULTURA':'https://www.diaridegirona.cat/cultura/',
    'OCI':'https://oci.diaridegirona.cat/',
    'TECNOLOGIA':'https://www.diaridegirona.cat/tecnologia/',
    'SALUT':'https://www.diaridegirona.cat/salut/'
}

In [7]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.106 Safari/537.36'
}

In [8]:
def get_text_from_url(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.text
        soup = BeautifulSoup(data, 'html.parser')
        return soup.select('span.cuerpo_texto100')[0].text.strip()

In [9]:
result= get_text_from_url('https://www.diaridegirona.cat/salut/2020/08/12/estudi-resol-dels-misteris-del/1057148.html')

In [10]:
result

'Un grup d\'investigadors liderats pel director de l\'Institut de Recerca contra la Leucèmia Josep Carreras, Manel Esteller, ha resolt un dels misteris del càncer amb l\'epigenètica, segons ha informat l\'institut. La investigació publicada a la revista \'Proceedings of the National Academy of Sciences\' apunta que en les cèl·lules canceroses s\'inactiva epigenèticament la proteïna que genera el nucleòtid \'Y\', originant tumors petits però altament agressius. "Hem construït el pont entre aquests dos móns al demostrar que el silenciament epigenètic del gen TYW2 és la causa de la pèrdua de l\'esquiu nucleòtid \'Y\'", segons destaca l\'article. Afegeix que el bloqueig epigenètic del gen TYW2 ocorre principalment en càncer de còlon, estómac i úter. El "misteri" data del 1975, quan \'Nature\' va publicar que s\'havia identificat un ARN entre els primers descobriments sobre les alteracions del material genètic als tumors al que faltava el nucleòtid \'Y\'.'

In [11]:
def get_urls_from_type():
    urls = []
    for id, url in URLs.items():
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            news = soup.select('div.noticia')
            for new in news: 
                a= new.select('a')
                if 'diaridegirona' not in a[0]['href']:
                    urls.append((id,"https://www.diaridegirona.cat{}".format((a[0]['href']))))
    return urls

In [12]:
URLS = get_urls_from_type()
number_of_words = 0
information = {}
pbar = tqdm(total=len(URLS))
for id, url in URLS:
    try:
        text = get_text_from_url(url)
        number_of_words += len(text)
        if id not in information:
            information[id] = []
        information[id].append(text)
    except Exception as e:
        print('Wrong url')
    pbar.update(1)

 31%|███▏      | 22/70 [00:06<00:10,  4.66it/s]

Wrong url
Wrong url


100%|██████████| 70/70 [00:19<00:00,  4.09it/s]

In [146]:
information

{'SPORTS': ["Una terrible calamarsada ha castigat aquest dijous els corredors més endarrerits que pujaven cap al cim del Col de Porte en la segona etapa del Critérium del Dauphiné que ha guanyat el ciclista eslovè Primoz Roglic. Diversos ciclistes s'han vist obligats a posar peu a terra, ja que el terra estava cobert de boles de gel. D'altres s'han refugiat on han pogut i alguns s'han presentat a l'autobús del seu equip amb marques a l'esquena pels cops de la calamarsa, tal com explica l'edició digital d'EL PERIÓDICO, del mateix grup editorial que Diari de GironaUn d'aquests ciclistes ha sigut el corredor francès Maxime Chevalier. El seu equip, el B&B Hotels-Vital Concept, ha distribuït a través del seu compte oficial una fotografia amb les ferides del seu corredor. Però no ha sigut l'únic afectat per culpa de la calamarsa. Un altre corredor amb marques a l'esquena ha estat Tim Declercq, del conjunt Deceunick.  ?? @maxxxchr attaqué par un peloton de grêlons sur la deuxième étape du @da

In [13]:
number_of_words

175779

In [13]:
total = 0
for id, news in information.items():
    total += len(news)
    print('{} has {} articles'.format(id, len(news)))
print('\nWe have a total of {} articles'.format(total))

SPORTS has 17 articles
ECONOMIA has 18 articles
CULTURA has 15 articles
TECNOLOGIA has 9 articles
SALUT has 9 articles

We have a total of 68 articles


In [14]:
with open('data.csv', 'w') as f:
    f.write('Type|Article\n')
    for key in information.keys():
        for new in information[key]:
            f.write('%s|%s\n'%(key,new))